# Exploration 14
## 프로젝트 - Movielens 영화 추천 실습
___

## 목차
- Step 1. 데이터 준비와 전처리
- Step 2. 분석해 보기
- Step 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가하기
- Step 4. CSR matrix를 직접 만들어 보기
- Step 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보기
- Step 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 에측한 나의 선호도 파악
- Step 7. 내가 좋아하는 영화와 비슷한 영화를 추천 받아보기
- Step 8. 내가 가장 좋아할 만한 영화들을 추천 받아보기
___

# Step 1. 데이터 준비와 전처리
사용할 데이터 
- 추천 시스템의 MNIST라고 부를만한 Movielens데이터 

조건 
- 별점 데이터는 대표적인 explicit 데이터이다. 하지만 implicit 데이터로 간주하고 테스트
- 별점을 시청횟수로 해석해서 생각
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### movie_id를 movie_name으로 바꾸기

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
data = pd.merge(ratings,movies, how='left', on=['movie_id'])
data.head(5)

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


# Step 2. 분석해 보기
- data에 있는 유니크한 영화 개수
- data에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

### data에 있는 유니크한 영화 개수

In [8]:
data['movie_id'].nunique()

3628

### data에 있는 유니크한 사용자 수

In [9]:
data['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [10]:
movie_cnt = data.groupby('title')['user_id'].count()
movie_cnt.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# Step 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가하기

### 좋아하는 영화 검색하기

검색의 효율성을 위해 'title'을 소문자로 바꾸기

In [11]:
data['title'] = data['title'].str.lower()
data.head(10)

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy
5,1,1197,3,978302268,"princess bride, the (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,978302039,ben-hur (1959),Action|Adventure|Drama
7,1,2804,5,978300719,"christmas story, a (1983)",Comedy|Drama
8,1,594,4,978302268,snow white and the seven dwarfs (1937),Animation|Children's|Musical
9,1,919,4,978301368,"wizard of oz, the (1939)",Adventure|Children's|Drama|Musical


검색하기
- American Beatuty
- Matrix
- Sixth Sense
- Men in Black
- Toy Story

In [12]:
# American Beatuty
data[data['title'].str.contains('american beauty')]['movie_id'].unique()

array([2858])

In [13]:
# Matrix
data[data['title'].str.contains('matrix')]['movie_id'].unique()

array([2571])

In [14]:
# Sixth Sense
data[data['title'].str.contains('sixth sense')]['movie_id'].unique()

array([2762])

In [15]:
# Men in Black
data[data['title'].str.contains('men in black')]['movie_id'].unique()

array([1580])

In [16]:
# Toy Story
data[data['title'].str.contains('toy story')]['movie_id'].unique()

array([   1, 3114])

### 항목의 추가하기

data에서 필요한 컬럼만 추출하기 ('user_id', 'title', 'count')

In [17]:
df_all = data.copy()

In [18]:
df = df_all[['user_id','title', 'counts']]
df.head()

,user_id,title,counts
0,1,one flew over the cuckoo's nest (1975),5
1,1,james and the giant peach (1996),3
2,1,my fair lady (1964),3
3,1,erin brockovich (2000),4
4,1,"bug's life, a (1998)",5


좋아하는 리스트를 만든후 추가하기

In [19]:
my_favorite = ['american beauty (1999)',
               'matrix, the (1999)',
               'sixth sense, the (1999)',
               'men in black (1997)',
               'toy story (1995)']

# 'jimin'이라는 user_id가 위의 영화를 3회씩 봤다고 가정
my_playlist = pd.DataFrame({'user_id': ['jimin']*5, 'title': my_favorite, 'counts':[3]*5})

if not df.isin({'user_id':['jimin']})['user_id'].any():  
    df = df.append(my_playlist)                           

df.tail(10)       

,user_id,title,counts
836473,6040,platoon (1986),3
836474,6040,"crying game, the (1992)",5
836475,6040,welcome to the dollhouse (1995),5
836476,6040,sophie's choice (1982),4
836477,6040,e.t. the extra-terrestrial (1982),4
0,jimin,american beauty (1999),3
1,jimin,"matrix, the (1999)",3
2,jimin,"sixth sense, the (1999)",3
3,jimin,men in black (1997),3
4,jimin,toy story (1995),3


# Step 4. CSR matrix를 직접 만들어 보기

### indexing 작업

In [20]:
# 고유한 유저, 타이틀를 찾아내는 코드
user_unique = df['user_id'].unique()
title_unique = df['title'].unique()

# 유저,타이틀 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jimin'])    

6039


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')
    
# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = df['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(df):
    print('title column indexing OK!!')
    df['title'] = temp_title_data
else:
    print('title column indexing Fail!!')


df

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,99,3
1,6039,124,3
2,6039,38,3
3,6039,175,3


### CSR matrix만들기

In [22]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['title'].nunique()

csr_data = csr_matrix((df.counts, (df.user_id, df.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [23]:
print(num_user, num_movie)

6040 3628


# Step 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보기
implicit패키지 사용하기

- implicit 패키지: 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지
- als (AlternatingLeastSquares): MF에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 방식

In [24]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### 모델 선언
AlternatingLeastSquares클래스의 __init__파라미터

- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미, 데이터를 몇 번 반복해서 학습할 것인지

In [25]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100,
                                    regularization=0.01,
                                    use_gpu=False,
                                    iterations=15,
                                    dtype=np.float32)

In [26]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

### 모델 훈련

In [27]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# Step 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악

### 훈련된 벡터 확인
- jimin 벡터와 american beauty의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는 지

In [35]:
jimin, american_beauty = user_to_idx['jimin'], title_to_idx['american beauty (1999)']
jimin_vector, american_beauty_vector = als_model.user_factors[jimin], als_model.item_factors[american_beauty]

In [34]:
jimin_vector

array([-0.5620288 , -0.3321715 , -0.470571  , -0.02560466, -0.38638577,
        0.10654359,  0.37115192, -0.21817039, -0.07924775, -0.02397361,
        0.00441864, -0.28401205, -0.05454424, -0.15182243, -0.09233445,
        0.0269881 ,  0.12320386,  0.32366866,  0.42046142, -0.03417464,
       -0.43339932,  0.09828121, -0.82396495, -0.50428486, -0.21794137,
       -0.7503733 ,  0.32609007,  0.9327339 ,  0.8297641 ,  1.0334305 ,
        0.3879757 ,  0.16795668, -0.4043372 , -1.0293367 ,  0.31358778,
       -0.349853  ,  0.19248666,  0.55412734,  0.7011725 , -0.06864824,
        0.09163382, -0.13739684,  0.25974357, -0.36661413,  0.16540368,
        0.02758536, -0.42321998,  0.25340492, -0.16235055, -0.12517361,
        0.04395785,  0.08882964,  0.35991657,  0.81146735,  0.02557564,
        0.30869868, -0.42161354, -0.4264734 ,  0.26426762,  0.39786386,
       -0.32268512, -0.5740247 ,  0.9085503 , -0.20478831,  0.5459317 ,
        0.24974965,  0.285974  ,  0.10397724,  0.7040491 ,  0.14

In [36]:
american_beauty_vector

array([-0.01870267,  0.0051495 ,  0.00584507, -0.00868476,  0.00607705,
        0.00588508, -0.00677337,  0.00917711,  0.02186581, -0.00221474,
        0.02736012, -0.03557993,  0.02618803,  0.02275003, -0.02965786,
       -0.02244477,  0.00372587,  0.0331361 ,  0.01098518, -0.00340004,
       -0.02099659,  0.0310675 , -0.02664924, -0.02845267,  0.01507611,
       -0.04298526,  0.02015938,  0.02309335,  0.03147266,  0.01637109,
        0.00124916, -0.01320861, -0.05018009, -0.0256887 ,  0.00111552,
        0.00213232, -0.01761759,  0.03163189,  0.0385392 ,  0.03004484,
       -0.02620233,  0.03137438,  0.01326573, -0.02953386, -0.01842892,
       -0.02322426, -0.03560068,  0.01595242, -0.02745167, -0.01790163,
        0.00464485,  0.03175325,  0.00378722,  0.00936243, -0.01346144,
        0.00807423, -0.00957995,  0.01119101,  0.0302101 ,  0.03445131,
       -0.00879731, -0.01747404,  0.05289544, -0.02436372,  0.03046355,
        0.03649105, -0.00435538,  0.00726703,  0.01380636, -0.01

In [37]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(jimin_vector, american_beauty_vector)

0.5572785

### 보지 못한 영화 데이터에 대한 선호도 예측
- 터미네이터

In [39]:
terminator = title_to_idx['terminator 2: judgment day (1991)']
terminator_vector = als_model.item_factors[terminator]
np.dot(jimin_vector, terminator_vector)

0.4362444

# Step 7. 내가 좋아하는 영화와 비슷한 영화를 추천 받아보기

### 비슷한 영화 추천 받기
- (movie_id, 유사도)

In [45]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

In [60]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [61]:
get_similar_movie('ghostbusters (1984)')

['ghostbusters (1984)',
 'who framed roger rabbit? (1988)',
 "ferris bueller's day off (1986)",
 'back to the future (1985)',
 'big (1988)',
 'beetlejuice (1988)',
 'fish called wanda, a (1988)',
 'airplane! (1980)',
 'bull durham (1988)',
 'gremlins (1984)']

# Step 8. 내가 가장 좋아할 만한 영화들을 추천 받아보기

### user에게 영화 추천하기

In [48]:
user = user_to_idx['jimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.43624437),
 (107, 0.4009194),
 (170, 0.34761345),
 (50, 0.3417632),
 (126, 0.28365567),
 (60, 0.27696043),
 (121, 0.26365232),
 (160, 0.25314593),
 (141, 0.24972682),
 (110, 0.23808624),
 (62, 0.23113385),
 (670, 0.23030472),
 (48, 0.22947189),
 (87, 0.220405),
 (64, 0.21874705),
 (271, 0.21615037),
 (44, 0.20165431),
 (51, 0.19135347),
 (22, 0.18774277),
 (273, 0.1767576)]

In [64]:
recommend_list = [idx_to_title[i[0]] for i in movie_recommended]
recommend_list

['terminator 2: judgment day (1991)',
 'jurassic park (1993)',
 'being john malkovich (1999)',
 'toy story 2 (1999)',
 'shakespeare in love (1998)',
 'star wars: episode i - the phantom menace (1999)',
 'silence of the lambs, the (1991)',
 'forrest gump (1994)',
 'fugitive, the (1993)',
 'groundhog day (1993)',
 'total recall (1990)',
 'galaxy quest (1999)',
 'saving private ryan (1998)',
 'braveheart (1995)',
 'star wars: episode vi - return of the jedi (1983)',
 'election (1999)',
 'star wars: episode iv - a new hope (1977)',
 'fargo (1996)',
 'back to the future (1985)',
 'fight club (1999)']

### 기여도 확인하기

In [72]:
def explain_rec(title):
    movie_id = title_to_idx[title]
    explain = als_model.explain(user, csr_data, itemid=movie_id)
    return [(idx_to_title[i[0]], i[1]) for i in explain[1]]

In [76]:
for title in recommend_list:
    print('\n',title)
    list = explain_rec(title)
    for li in list:
        print(li)


 terminator 2: judgment day (1991)
('matrix, the (1999)', 0.19213259809854372)
('men in black (1997)', 0.14951537833135078)
('sixth sense, the (1999)', 0.040538752872259914)
('american beauty (1999)', 0.02677621899128993)
('toy story (1995)', 0.020442154399288898)

 jurassic park (1993)
('men in black (1997)', 0.23496304370258073)
('matrix, the (1999)', 0.1117330548452071)
('american beauty (1999)', 0.04364302646249446)
('sixth sense, the (1999)', 0.02835575237588913)
('toy story (1995)', -0.02344710226001663)

 being john malkovich (1999)
('american beauty (1999)', 0.2525968691176892)
('sixth sense, the (1999)', 0.1150986355779332)
('toy story (1995)', 0.029354553379260254)
('matrix, the (1999)', -0.013690668480403706)
('men in black (1997)', -0.04091648468990499)

 toy story 2 (1999)
('toy story (1995)', 0.31598505987356545)
('sixth sense, the (1999)', 0.03179803432847853)
('matrix, the (1999)', 0.004988487392346569)
('american beauty (1999)', -0.0030006347018830875)
('men in black 

___
# 프로젝트 정리 

## 회고
- 어제 fundamental노드로 추천시스템을 배웠을때 흥미로웠는데 실습적인 부분이 아쉬웠습니다. 근데 오늘 실습을 통해 추천시스템을 더욱 잘 이해한것 같고 재미있었습니다.
- 또한 내가 좋아하는 영화를 잘 추천해준것도 신기한데 나의 어떤 부분에서 이것을 추천해줬는지(explain)를 알려주어서 신기했습니다

## 결과
- 마지막에 추천해준 것들을 분석한 결과 

추천영화| american beauty|sixth sense|matrix|toy story|men in black
---|---|---|---|---|---|
terminator 2: judgment day (1991)|0.02677621899128993|0.040538752872259914|0.19213259809854372|0.020442154399288898|0.14951537833135078
jurassic park (1993)|0.04364302646249446|0.02835575237588913|0.1117330548452071|-0.02344710226001663|0.23496304370258073
 being john malkovich (1999)|0.2525968691176892|0.1150986355779332|-0.013690668480403706|0.029354553379260254|-0.04091648468990499
shakespeare in love (1998)|0.11587419614207417|0.058225720121696356|0.008312437148134783|0.08262378056236212|0.01309364022783
star wars: episode i - the phantom menace (1999)|0.054325082234792446|0.07187639635295837|0.047992333492273116|0.010058830293764063|0.0863694350442617

- 다음과 같이 상위 5개만 추려봤을때 유사도에 측면에서 모델이 잘 동작하는것을 확인 할 수 있었습니다.